In [1]:
%load_ext watermark

In [2]:
%watermark

2017-05-15T22:36:49-04:00

CPython 3.6.0
IPython 5.3.0

compiler   : GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)
system     : Darwin
release    : 16.5.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


In [3]:
from tf_idf import preprocess_fundamentos, remove_accents
from sklearn import model_selection
from tqdm import tqdm, trange
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter, defaultdict
from operator import itemgetter

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import UDP
import logging
import random
import pandas as pd
import numpy as np

from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC  # support vector machine classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  # naive bayes
from sklearn.neighbors import KNeighborsClassifier

# Setup

In [4]:
logger = logging.getLogger(__name__)

engine = create_engine('mysql://root@localhost/constabierta')
_session = sessionmaker(bind=engine, autocommit=True, expire_on_commit=True)
session = _session()

documents = session.query(UDP).all()
SEED = 37

In [5]:
docs_df = pd.DataFrame({'fundamento': [doc.fundamento for doc in documents],
                        'label': [remove_accents(doc.modo.strip().lower()) for doc in documents]})

# Análisis Exploratorio 1

In [6]:
docs_df.shape

(51867, 2)

In [7]:
pd.options.display.max_colwidth = 0
k = 5
docs_df.sample(k)

,fundamento,label
33617,"Son la esperanza del mundo, las futuras capacidades que con una buen cuidado y desarrollo, son una herramienta crucial para el desarrollo de la Sociedad. En Chile actualmente hay muchos niños abandonados y maltratados por su familia y la sociedad chilena no es capaz de darle su debida protección",factico.definicion
29428,"PARTICIPACION REAL E IGUALITARIA, UN ESTADO SIN SISTEMA BINOMINAL.",propositivo
42892,Esencial para la dignidad del ser humano,valorativo.pragmatico
13921,"democratización de la justicia, igualdad de ley en educación, igualdad en el derecho a la tierra, igualdad sin importar origen, derechos sociales y ambientales",propositivo
46012,"Para poder elegir que tipo de educación queremos para nuestros hijos, que no solo sea la estructura que nos entrega el estado.",valorativo.pragmatico


In [8]:
docs_df['label'].value_counts()

propositivo              34501
factico.definicion       8999 
indeterminado            3974 
valorativo.pragmatico    2066 
blanco                   1578 
factico.causalidad       388  
factico.prediccion       228  
valorativo.negativo      55   
valorativo.positivo      45   
factico.pasado           31   
factico.negativo         1    
valorativa.pragmatica    1    
Name: label, dtype: int64

Corregir `valorativa.pragmatica` a `valorativo.pragmativo`

In [9]:
docs_df[docs_df['label'] == 'valorativa.pragmatica']

,fundamento,label
33636,"etapa fundamental del ciclo vital la cual es decisiva para las dinámicas sociales, políticas, culturales. Permite asegurar la mejor calidad de vida.",valorativa.pragmatica


In [10]:
docs_df = docs_df.set_value(33636, 'label', 'valorativo.pragmatico')
print(docs_df[docs_df['label'] == 'valorativa.pragmatica'])
print('👍')

Empty DataFrame
Columns: [fundamento, label]
Index: []
👍


In [11]:
docs_df['label'].value_counts()

propositivo              34501
factico.definicion       8999 
indeterminado            3974 
valorativo.pragmatico    2067 
blanco                   1578 
factico.causalidad       388  
factico.prediccion       228  
valorativo.negativo      55   
valorativo.positivo      45   
factico.pasado           31   
factico.negativo         1    
Name: label, dtype: int64

Agrupar labels por primer campo (propositivo, factico, indeterminado, valorativo, blanco)

In [12]:
docs_df['group'] = docs_df['label'].apply(lambda x: x.split('.')[0])
docs_df['group'].value_counts()

propositivo      34501
factico          9647 
indeterminado    3974 
valorativo       2167 
blanco           1578 
Name: group, dtype: int64

In [13]:
k = 5
rows = np.random.choice(docs_df.index.values, 5)
docs_df.ix[rows]

,fundamento,label,group
43922,5 votos.,blanco,blanco
22950,No podemos disponer de los recursos de la naturaleza como si nos pertenecieran. Existe una desconexión del ser humano con la naturaleza y es derecho de toda persona vivir en un ambiente limpio y no vulnerado.,indeterminado,indeterminado
41638,"Enfoque de equidad, sensibilidad de género y diversidad. Equidad socioeconómica: que todas las personas tengan el mismo acceso a este derecho y otros. Que se fortalezca el sistema público de salud y se unifique en un sistema nacional y estatal que garantice la vida digna.",propositivo,propositivo
27184,Salario y condiciones laborales dignas. Todos tienen derecho a trabajar en buenas condiciones y recibir un buen salario para poder llevar una vida digna con el fin de vivir y no sobrevivir.,propositivo,propositivo
14256,"A la vida, integridad física y psíquica. Derecho a la vida desde la concepción hasta la muerte natural.",propositivo,propositivo


## Procesar fundamentos

In [14]:
# esto demora 5 minutos aprox
%time X, processed_texts, y, le = preprocess_fundamentos(docs_df, group=True)

100%|██████████| 51867/51867 [05:03<00:00, 171.18it/s]

CPU times: user 4min 38s, sys: 21.6 s, total: 5min
Wall time: 5min 3s


In [15]:
X.shape, y.shape

((51867, 13222), (51867,))

In [16]:
docs_df['texto_procesado'] = processed_texts
docs_df['y'] = y

In [1]:
docs_df.groupby('group').apply(lambda x: x.sample(5))[['fundamento', 'texto_procesado']]

NameError: name 'docs_df' is not defined

## Vocabulario

In [18]:
vocab = Counter()
for tokens in tqdm(processed_texts):
    for token in tokens:
        vocab[token] += 1

100%|██████████| 51867/51867 [00:00<00:00, 138163.90it/s]


In [19]:
k = 10
tmp = pd.DataFrame({'word': list(map(itemgetter(0), vocab.most_common(k))),
                    'freq': list(map(itemgetter(1), vocab.most_common(k)))})
tmp

,freq,word
0,25495,derech
1,19617,deb
2,9281,ser
3,7857,person
4,7695,salud
5,7320,calid
6,6718,garantiz
7,6709,educ
8,5959,vid
9,5876,tod


# Clasificación

In [20]:
total = sum(Counter(y).values())

for label, count in Counter(y).most_common():
    print(label, count, '%.5f' % (count / total), le.inverse_transform([label]))

3 34501 0.66518 ['propositivo']
1 9647 0.18599 ['factico']
2 3974 0.07662 ['indeterminado']
4 2167 0.04178 ['valorativo']
0 1578 0.03042 ['blanco']


In [21]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    test_size=.33, 
                                                                    random_state=SEED, 
                                                                    stratify=y)

In [22]:
target_names = le.inverse_transform(range(max(y) + 1)).tolist()
target_names

['blanco', 'factico', 'indeterminado', 'propositivo', 'valorativo']

In [23]:
print("y distribution")
print(pd.Series(y).value_counts(normalize=True))
print()
print("y_train distribution")
print(pd.Series(y_train).value_counts(normalize=True))
print()
print("y_test distribution")
print(pd.Series(y_test).value_counts(normalize=True))

y distribution
3    0.665182
1    0.185995
2    0.076619
4    0.041780
0    0.030424
dtype: float64

y_train distribution
3    0.665180
1    0.185986
2    0.076633
4    0.041784
0    0.030417
dtype: float64

y_test distribution
3    0.665187
1    0.186014
2    0.076591
4    0.041771
0    0.030438
dtype: float64


In [32]:
X_train_dense = X_train.todense()
X_test_dense = X_test.todense()
X_dense = X.todense()

In [33]:
#### los archivos resultantes son muy grandes para weka
with open('train.csv', 'w') as f:
    for row, label in zip(X_train_dense, y_train):
        f.write(','.join(map(str, row.tolist()[0])) + ',' + le.inverse_transform(label) + '\n')

with open('test.csv', 'w') as f:
    for row, label in zip(X_test_dense, y_test):
        f.write(','.join(map(str, row.tolist()[0])) + ',' + le.inverse_transform(label) + '\n')

with open('full.csv', 'w') as f:
    for row, label in zip(X_dense, y):
        f.write(','.join(map(str, row.tolist()[0])) + ',' + le.inverse_transform(label) + '\n')

In [ ]:
c0 = ("Base", DummyClassifier(strategy='stratified'))
c1 = ("SVM", SVC(kernel='rbf'))
c2 = ("SVM", SVC(kernel='linear'))
c3 = ("DT", DecisionTreeClassifier())
c4 = ("NB", GaussianNB())
c5 = ("KNN", KNeighborsClassifier(n_neighbors=5))

clfs = [c0, c1, c2, c3, c4, c5]

for name, clf in clfs:
    print(name)
    clf.fit(X_train_d, y_train)
    y_pred = clf.predict(X_test_d)
    print(classification_report(y_test, y_pred, target_names=target_names))

Base
               precision    recall  f1-score   support

       blanco       0.03      0.03      0.03       521
      factico       0.18      0.18      0.18      3184
indeterminado       0.07      0.08      0.08      1311
  propositivo       0.66      0.67      0.67     11386
   valorativo       0.03      0.03      0.03       715

  avg / total       0.48      0.49      0.49     17117

SVM


/Users/mquezada/anaconda3/envs/ams/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


               precision    recall  f1-score   support

       blanco       0.00      0.00      0.00       521
      factico       0.00      0.00      0.00      3184
indeterminado       0.00      0.00      0.00      1311
  propositivo       0.67      1.00      0.80     11386
   valorativo       0.00      0.00      0.00       715

  avg / total       0.44      0.67      0.53     17117

SVM
               precision    recall  f1-score   support

       blanco       0.57      0.41      0.48       521
      factico       0.55      0.25      0.35      3184
indeterminado       0.42      0.03      0.05      1311
  propositivo       0.73      0.97      0.83     11386
   valorativo       0.57      0.02      0.04       715

  avg / total       0.66      0.71      0.64     17117

DT
               precision    recall  f1-score   support

       blanco       0.48      0.72      0.58       521
      factico       0.38      0.35      0.36      3184
indeterminado       0.15      0.09      0.11      1